In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install altair vega_datasets

In [3]:
import pandas as pd
import numpy as np
import altair as alt
from vega_datasets import data

In [4]:
BASE_DIR = "/content/drive/MyDrive/BMI_706"
hiv_path = f"{BASE_DIR}/TB_hiv_nonroutine_surveillance_2025-11-18.csv"
hiv = pd.read_csv(hiv_path)
hiv.head()

,country,iso2,iso3,iso_numeric,g_whoregion,year,tbhiv_surv_prev,tbhiv_surv_yr,tbhiv_surv_cil,tbhiv_surv_ciu,tbhiv_sentin_prev,tbhiv_sentin_yr,tbhiv_sentin_cil,tbhiv_sentin_ciu
0,Algeria,DZ,DZA,12,AFR,2007,0.18,2002.0,NaN,NaN,NaN,NaN,NaN,NaN
1,American Samoa,AS,ASM,16,WPR,2008,100.00,2008.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,AO,AGO,24,AFR,2009,NaN,NaN,NaN,NaN,9.0,2007.0,NaN,NaN
3,Angola,AO,AGO,24,AFR,2011,NaN,NaN,NaN,NaN,14.0,2011.0,NaN,NaN
4,Anguilla,AI,AIA,660,AMR,2013,0.00,2013.0,0.0,0.0,0.0,2013.0,0.0,0.0


In [5]:
hiv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   country            286 non-null    object 
 1   iso2               283 non-null    object 
 2   iso3               286 non-null    object 
 3   iso_numeric        286 non-null    int64  
 4   g_whoregion        286 non-null    object 
 5   year               286 non-null    int64  
 6   tbhiv_surv_prev    175 non-null    float64
 7   tbhiv_surv_yr      177 non-null    float64
 8   tbhiv_surv_cil     86 non-null     float64
 9   tbhiv_surv_ciu     83 non-null     float64
 10  tbhiv_sentin_prev  182 non-null    float64
 11  tbhiv_sentin_yr    176 non-null    float64
 12  tbhiv_sentin_cil   94 non-null     float64
 13  tbhiv_sentin_ciu   95 non-null     float64
dtypes: float64(8), int64(2), object(4)
memory usage: 31.4+ KB


In [6]:
len(hiv['country'].unique())

114

In [7]:
#Survey-only records
survey = hiv.dropna(subset=['tbhiv_surv_prev']).copy()

survey = (
    survey
    .sort_values(['iso_numeric', 'tbhiv_surv_yr'], na_position='first')
    .drop_duplicates(subset='iso_numeric', keep='last')
)

survey['mode'] = 'Survey'
survey['prev'] = survey['tbhiv_surv_prev']
survey['data_year'] = survey['tbhiv_surv_yr']
survey['source_type'] = 'Survey'

survey.head()

,country,iso2,iso3,iso_numeric,g_whoregion,year,tbhiv_surv_prev,tbhiv_surv_yr,tbhiv_surv_cil,tbhiv_surv_ciu,tbhiv_sentin_prev,tbhiv_sentin_yr,tbhiv_sentin_cil,tbhiv_sentin_ciu,mode,prev,data_year,source_type
0,Algeria,DZ,DZA,12,AFR,2007,0.18,2002.0,NaN,NaN,NaN,NaN,NaN,NaN,Survey,0.18,2002.0,Survey
1,American Samoa,AS,ASM,16,WPR,2008,100.00,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,Survey,100.00,2008.0,Survey
6,Antigua and Barbuda,AG,ATG,28,AMR,2009,0.00,2009.0,NaN,NaN,NaN,NaN,NaN,NaN,Survey,0.00,2009.0,Survey
11,Barbados,BB,BRB,52,AMR,2009,0.00,2008.0,0.0,0.0,0.0,2008.0,0.0,0.0,Survey,0.00,2008.0,Survey
16,Botswana,BW,BWA,72,AFR,2008,67.50,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,Survey,67.50,2008.0,Survey


In [8]:
#Sentinel-only records
sentinel = hiv.dropna(subset=['tbhiv_sentin_prev']).copy()

sentinel = (
    sentinel
    .sort_values(['iso_numeric', 'tbhiv_sentin_yr'], na_position='first')
    .drop_duplicates(subset='iso_numeric', keep='last')
)

sentinel['mode'] = 'Sentinel'
sentinel['prev'] = sentinel['tbhiv_sentin_prev']
sentinel['data_year'] = sentinel['tbhiv_sentin_yr']
sentinel['source_type'] = 'Sentinel'

sentinel.head()

,country,iso2,iso3,iso_numeric,g_whoregion,year,tbhiv_surv_prev,tbhiv_surv_yr,tbhiv_surv_cil,tbhiv_surv_ciu,tbhiv_sentin_prev,tbhiv_sentin_yr,tbhiv_sentin_cil,tbhiv_sentin_ciu,mode,prev,data_year,source_type
3,Angola,AO,AGO,24,AFR,2011,NaN,NaN,NaN,NaN,14.0,2011.0,NaN,NaN,Sentinel,14.0,2011.0,Sentinel
8,Antigua and Barbuda,AG,ATG,28,AMR,2018,NaN,NaN,NaN,NaN,0.0,2018.0,NaN,NaN,Sentinel,0.0,2018.0,Sentinel
10,Bahamas,BS,BHS,44,AMR,2009,NaN,NaN,NaN,NaN,33.0,2009.0,19.5,48.0,Sentinel,33.0,2009.0,Sentinel
11,Barbados,BB,BRB,52,AMR,2009,0.0,2008.0,0.0,0.0,0.0,2008.0,0.0,0.0,Sentinel,0.0,2008.0,Sentinel
246,Solomon Islands,SB,SLB,90,WPR,2007,0.0,NaN,NaN,NaN,0.0,2007.0,NaN,NaN,Sentinel,0.0,2007.0,Sentinel


In [9]:
#Combined: prefer survey, else sentinel, one observation per country
## 1. if survey, then survey (already in the most recent year)
## 2. if no survey, then sentinel (already in the most recent year)
## 3. if no survey nor sentinel, remove country from combined dataset

combined_from_survey = survey.copy()
combined_from_survey['mode'] = 'Combined'

survey_iso = set(survey['iso_numeric'])
sentinel_only = sentinel[~sentinel['iso_numeric'].isin(survey_iso)].copy()

combined_from_sentinel = sentinel_only.copy()
combined_from_sentinel['mode'] = 'Combined'

combined = pd.concat(
    [combined_from_survey, combined_from_sentinel],
    ignore_index=True
)

In [10]:
#Merge into one long table
cols_keep = ['country', 'iso_numeric', 'g_whoregion', 'prev', 'data_year', 'source_type', 'mode']

hiv_long = pd.concat(
    [survey[cols_keep], sentinel[cols_keep], combined[cols_keep]],
    ignore_index=True
)

hiv_long.head()

,country,iso_numeric,g_whoregion,prev,data_year,source_type,mode
0,Algeria,12,AFR,0.18,2002.0,Survey,Survey
1,American Samoa,16,WPR,100.00,2008.0,Survey,Survey
2,Antigua and Barbuda,28,AMR,0.00,2009.0,Survey,Survey
3,Barbados,52,AMR,0.00,2008.0,Survey,Survey
4,Botswana,72,AFR,67.50,2008.0,Survey,Survey


In [11]:
hiv_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   country      287 non-null    object 
 1   iso_numeric  287 non-null    int64  
 2   g_whoregion  287 non-null    object 
 3   prev         287 non-null    float64
 4   data_year    264 non-null    float64
 5   source_type  287 non-null    object 
 6   mode         287 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 15.8+ KB


In [12]:
hiv_long['mode'].value_counts()

,count
mode,
Combined,113
Survey,94
Sentinel,80


In [13]:
len(hiv_long['country'].unique())

113

In [14]:
hiv_long.isna().sum()

,0
country,0
iso_numeric,0
g_whoregion,0
prev,0
data_year,23
source_type,0
mode,0


In [15]:
hiv_long['data_year'] = (
    hiv_long['data_year']
    .fillna("Unknown")
    .astype(str)
    .str.replace('.0', '', regex=False)
)

hiv_long.head()

,country,iso_numeric,g_whoregion,prev,data_year,source_type,mode
0,Algeria,12,AFR,0.18,2002,Survey,Survey
1,American Samoa,16,WPR,100.00,2008,Survey,Survey
2,Antigua and Barbuda,28,AMR,0.00,2009,Survey,Survey
3,Barbados,52,AMR,0.00,2008,Survey,Survey
4,Botswana,72,AFR,67.50,2008,Survey,Survey


In [16]:
world_map = alt.topo_feature(data.world_110m.url, 'countries')

def make_dashboard(mode='Combined'):
    df_mode = hiv_long[hiv_long['mode'] == mode].copy()

    region_select = alt.selection_point(
        fields=['g_whoregion'],
        name=f'RegionSel_{mode}'
    )

    map_chart = (
        alt.Chart(world_map)
        .mark_geoshape(stroke='white', strokeWidth=0.5)
        .transform_lookup(
            lookup='id',
            from_=alt.LookupData(
                df_mode,
                key='iso_numeric',
                fields=['country', 'g_whoregion', 'prev', 'data_year', 'source_type']
            )
        )
        .encode(
            color=alt.condition(
                alt.datum.prev != None,
                alt.Color(
                    'prev:Q',
                    title=f"HIV/TB prevalence (%) — {mode}",
                    scale=alt.Scale(scheme='oranges')
                ),
                alt.value('lightgray')
            ),
            opacity=alt.condition(
                region_select,
                alt.value(1.0),
                alt.value(0.4)
            ),
            tooltip=[
                alt.Tooltip('country:N',     title='Country'),
                alt.Tooltip('g_whoregion:N', title='WHO Region'),
                alt.Tooltip('prev:Q',        title='Prevalence (%)', format='.1f'),
                alt.Tooltip('source_type:N', title='Source'),
                alt.Tooltip('data_year:N',   title='Data year'),
            ]
        )
        .properties(
            width=520,
            height=280,
            title=f"HIV/TB Co-Infection Prevalence by Country ({mode})"
        )
        .project('equalEarth')
        .add_params(region_select)
    )

    box_chart = (
        alt.Chart(df_mode)
        .mark_boxplot(outliers=True)
        .encode(
            x=alt.X('g_whoregion:N', title='WHO Region'),
            y=alt.Y('prev:Q', title='HIV/TB prevalence (%)'),
            color=alt.condition(
                region_select,
                alt.Color('g_whoregion:N', title='WHO Region'),
                alt.value('lightgray')
            )
        )
        .properties(
            width=520,
            height=260,
            title=f"HIV/TB Prevalence by WHO Region ({mode})"
        )
    )

    return (map_chart & box_chart).resolve_scale(color='independent')

In [17]:
make_dashboard('Combined')

alt.VConcatChart(...)

In [18]:
make_dashboard('Survey')

alt.VConcatChart(...)

In [19]:
make_dashboard('Sentinel')

alt.VConcatChart(...)

In [20]:
hiv_long.to_csv("hiv_long.csv", index=False)